# Dimensionality reduction

In [58]:
from config import *

In [59]:
#importing libraries
from random import randint
import time
from time import sleep
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
import numpy as np
import pandas as pd

In [60]:
#establish the connection
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id=client_id,
                                                           client_secret=client_secret))

In [61]:
#Sklearn
from sklearn.datasets import make_classification
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from scipy.spatial.distance import euclidean
from sklearn.metrics import silhouette_score, calinski_harabasz_score
from kneed import KneeLocator
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc
import time
import pickle

### Read .csv

In [64]:
hot_songs_features = pd.read_csv('hot_songs_features.csv')
not_hot_songs_features = pd.read_csv('not_hot_songs_features.csv')

_____

## Concat DataFrame 


In [68]:
read = pd.concat([hot_songs_features, not_hot_songs_features], axis=0) 
read.head(1)

,Title,Artist,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,type,id,uri,track_href,analysis_url,duration_ms,time_signature
0,Paint The Town Red,Doja Cat,0.864,0.556,2.0,-7.683,0.0,0.194,0.255,0.000004,0.112,0.726,99.974,audio_features,56y1jOTK0XSvJzVv9vHQBK,spotify:track:56y1jOTK0XSvJzVv9vHQBK,https://api.spotify.com/v1/tracks/56y1jOTK0XSv...,https://api.spotify.com/v1/audio-analysis/56y1...,230480.0,4.0


In [69]:
read.shape

(2398, 20)

In [78]:
read.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2398 entries, 0 to 2297
Data columns (total 20 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Title             2398 non-null   object 
 1   Artist            2398 non-null   object 
 2   danceability      2393 non-null   float64
 3   energy            2393 non-null   float64
 4   key               2393 non-null   float64
 5   loudness          2393 non-null   float64
 6   mode              2393 non-null   float64
 7   speechiness       2393 non-null   float64
 8   acousticness      2393 non-null   float64
 9   instrumentalness  2393 non-null   float64
 10  liveness          2393 non-null   float64
 11  valence           2393 non-null   float64
 12  tempo             2393 non-null   float64
 13  type              2393 non-null   object 
 14  id                2393 non-null   object 
 15  uri               2393 non-null   object 
 16  track_href        2393 non-null   object 
 17  

In [81]:
features = read.iloc[:, list(range(2, 13)) + list(range(18, 20))].copy()
features.head(1)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.864,0.556,2.0,-7.683,0.0,0.194,0.255,0.000004,0.112,0.726,99.974,230480.0,4.0


## Scaling features


In [82]:
features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000,2393.000000
mean,0.654972,0.682573,5.246135,-6.050226,0.611366,0.097543,0.175801,0.029112,0.174655,0.530481,121.136483,223321.473046,3.974091
std,0.143712,0.176478,3.594404,2.677144,0.487542,0.092555,0.228216,0.136587,0.135874,0.231151,27.848254,44775.877261,0.252408
min,0.162000,0.009370,0.000000,-34.840000,0.000000,0.022500,0.000013,0.000000,0.021000,0.000010,50.141000,62253.000000,1.000000
25%,0.562000,0.576000,2.000000,-7.054000,0.000000,0.038000,0.018500,0.000000,0.090700,0.354000,99.050000,197547.000000,4.000000
50%,0.665000,0.706000,5.000000,-5.613000,1.000000,0.057300,0.076100,0.000000,0.120000,0.536000,120.040000,220291.000000,4.000000
75%,0.757000,0.816000,8.000000,-4.381000,1.000000,0.113000,0.237000,0.000064,0.222000,0.717000,138.015000,245000.000000,4.000000
max,0.975000,0.988000,11.000000,-0.276000,1.000000,0.591000,0.995000,0.991000,0.971000,0.974000,210.857000,688453.000000,5.000000


In [85]:
# Initialize the model, fit and transform data
scaler = StandardScaler()
scaler.fit(features)
features_scaled = scaler.transform(features)
features_scaled_df = pd.DataFrame(features_scaled, columns = features.columns)

In [86]:
features_scaled_df.head(3)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1.454791,-0.717367,-0.903297,-0.610021,-1.254240,1.042370,0.347109,-0.213159,-0.461220,0.846027,-0.760080,0.159908,0.102668
1,-0.667945,-0.745705,-0.068491,-0.441149,0.797295,0.372360,-0.152522,-0.213187,-0.475943,-0.599216,0.785546,-0.480749,0.102668
2,-0.723624,0.671201,0.209777,0.366596,0.797295,-0.726673,-0.713511,-0.213143,0.665060,0.408992,0.247483,0.389544,0.102668


### PCA

### ISOMAP

### TSNE

# Clustering

### K-MEAN

### DBSCAN

### HDBSCAN

In [ ]:
Metrics of clustering:
    - what is a cluster for you?